## Welcome to my project where I build an Agentic AI with No Framework.

Use Case: I am going to use my LinkedIN profile and Summary as inputs to the LLMs. Create an UI using Gradio package for user to be able to interact using natuarl langague and find out about me based on inputs I shared.

In the folder `MyResource` I've put a single file `LinkedinProfile.pdf` - it's a PDF download of my LinkedIn profile.

I've also made a file called `AboutMe.txt`

This is more like a foundation for AgenticAI to work without a framework in place

Note: Please note that i have changed the model name gemini-2.5-flash instead of gemini-2.0-flash

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#00bfff;">What packages?</h2>
            <span style="color:#00bfff;">In this project, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# If you don't know what any of these packages do - you can always ask Gemini for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr


In [4]:
import os
load_dotenv(override=True)

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
google_api_key = os.getenv("GOOGLE_API_KEY")
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)


In [5]:
reader = PdfReader("MyResources/LinkedinProfile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

   
Contact
pachipoola.hari@gmail.com
www.linkedin.com/in/
haripachipoola (LinkedIn)
Top Skills
Data Modeling
Digital Transformation
AI Strategy
Languages
Spanish
Hindi
Telugu
Certifications
Certified Ethical Hacker
Scrum Master Certificate of
Proficiency
Deep Learning Fundamentals
Introduction to Containers,
Kubernetes, and OpenShift
Exploratory Testing Specialist
Honors-Awards
Most Valuable Performer Award
Leadership and Value Recognition
Award
Voyager
Star Award
Honor Roll
Publications
SDET Process implementation
Hari Babu P
Global Delivery Leader | Transforming Operations into Centers of
Value | Data & AI Strategy
Santa Clara, California, United States
Summary
Visionary Technology Leader with 20+ years of experience
orchestrating high-value digital and data transformations for global
Financial and Healthcare institutions. Expert in navigating the
intersection of Technology Strategy, Financial Governance, and
Global Operations, scaling delivery organizations from inception to
300+ m

In [7]:
with open("MyResources/Aboutme.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Hari Babu Pachipoola"

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [10]:
system_prompt

"You are acting as Hari Babu Pachipoola. You are answering questions on Hari Babu Pachipoola's website, particularly questions related to Hari Babu Pachipoola's career, background, skills and experience. Your responsibility is to represent Hari Babu Pachipoola for interactions on the website as faithfully as possible. You are given a summary of Hari Babu Pachipoola's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nI am a global delivery leader for Data and AI. I have experience working with large programs and teams with team sizes of 200+ associates and annual run rate of $90M. I bring a blend of Delivery, operations with technical acumen and responsible for building several data modernization, migration, real time intelligence and agentic ai solutions. I had privilege to work with CXOs and have ex

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [15]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\pachi\projects\agents\.venv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pachi\projects\agents\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pachi\projects\agents\.venv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pachi\projects\agents\.venv\Lib\site-packages\gradio\blocks.py", line 1621, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pachi\projects\agents\.venv\Lib\site-packages\gradio\utils.py", line 882, in async_wrapper
    response = await f(*args, **kwargs)
    

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [23]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [24]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [25]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [26]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [27]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "do you hold a patent?", messages[:1])

In [30]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [35]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()